<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_websocket_server_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install achatbot

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev


In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server,ngrok_proxy,websocket_server_transport,silero_vad_analyzer,sense_voice_asr,openai,google_ai,together_ai,tts_edge,queue]"


# init api key

run bot task woker with bot.json, e.g.: dummy_bot.json

- use websocket protocol audio stream
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
- use image gen model:
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
  - [huggingface api](https://huggingface.co/docs/api-inference/tasks/text-to-image) HF_API_KEY




In [ ]:
from google.colab import userdata
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')


# download model

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False


# run bot with config

In [ ]:
!cat /content/websocket_server_bot.json

In [ ]:
!TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.bots.main -f /content/websocket_server_bot.json

# http fastapi

ui: https://github.com/ai-bot-pro/nextjs-react-web-storytelling

## use colab kernel proxy
just use in google chrome,
need cookie

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [ ]:
# https://m08wf4l3jc-496ff2e9c6d22116-8000-colab.googleusercontent.com/docs
# NOTE: just use in google chrome
# for http api server on colab
!TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.websocket.server.fastapi_ws_bot_serve --port 8000


## use ngrok proxy

In [ ]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')

In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN

In [18]:
!cat config/bots/fastapi_websocket_server_bot.json

{
  "chat_bot_name": "FastapiWebsocketServerBot",
  "transport_type": "websocket",
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "together",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2-72B-Instruct",
      "language": "zh",
      "messages": [
        {
          "role": "system",
          "content": "你是一名叫奥利给的智能助理。保持回答简短和清晰。请用中文回答。"
        }
      ]
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",
        "language": "zh",
        "gender": "Male"
      }
    }
  },
  "config_list": []
}


In [20]:
!TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.websocket.server.fastapi_ws_bot_serve --port 8000 --ngrok 1

2024-11-05 16:36:37,798 - pyngrok.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py:283 - connect - Opening tunnel named: http-8000-f436f5c2-93af-413b-a632-9d7486fe2bb4
2024-11-05 16:36:37,821 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-05T16:36:37+0000 lvl=info msg="no configuration paths supplied"
2024-11-05 16:36:37,822 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-05T16:36:37+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2024-11-05 16:36:37,822 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.py:99 - _log_line - t=2024-11-05T16:36:37+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
2024-11-05 16:36:37,835 - pyngrok.process.ngrok - INFO - /usr/local/lib/python3.10/dist-packages/pyngrok/process.p

# bot task queue redis

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')


In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json